![Py4Eng](img/logo.png)

# Numerical Python: NumPy
## Yoav Ram

[NumPy](http://www.numpy.org/) is the fundamental package for scientific computing with Python. It contains arrays, math functions, linear algebra, random number capabilities and much more.

# [![Numpy logo](https://numfocus.org/wp-content/uploads/2016/07/numpy-logo-300.png)](https://matplotlib.org/gallery/mplot3d/voxels_numpy_logo.html)

# Importing NumPy

The convention is `import numpy as np`. This loads the entire NumPy package once, and uses an alias `np` so that we don't pollute our code with too much `numpy`.

Some people like to do `from numpy import *`. This is frowned-upon as it pollutes the namespace: it overrides the default `sum` and hides the fact that we are using specific `numpy` functions.

If you only need specific NumPy objects you can load them using `from numpy import array, ones` etc.

In [1]:
import numpy as np
print("Numpy version:", np.__version__)

Numpy version: 1.15.3


# Analyzing Patient Data

We are studying inflammation in patients who have been given a new treatment for arthritis, and need to analyze the first dozen data sets. The data sets are stored in comma-separated values (CSV) format: each row holds information for a single patient, and the columns represent successive days. The first few rows of our data file look like this:

> 0,0,1,3,1,2,4,7,8,3,3,3,10,5,7,4,7,7,12,18,6,13,11,11,7,7,4,6,8,8,4,4,5,7,3,4,2,3,0,0
0,1,2,1,2,1,3,2,2,6,10,11,5,9,4,4,7,16,8,6,18,4,12,5,12,7,11,5,11,3,3,5,4,4,5,5,1,1,0,1
0,1,1,3,3,2,6,2,5,9,5,7,4,5,4,15,5,11,9,10,19,14,12,17,7,12,11,7,4,2,10,5,4,2,2,3,2,2,1,1
0,0,2,0,4,2,2,1,6,7,10,7,9,13,8,8,15,10,10,7,17,4,4,7,6,15,6,4,9,11,3,5,6,3,3,4,2,3,2,1
0,1,1,3,3,1,3,5,2,4,4,7,6,5,3,10,8,10,6,17,9,14,9,7,13,9,12,6,7,7,9,6,3,2,2,4,2,0,1,1


## Loading data from file

We read the file into a NumPy array - the new data structure which is the center of all scientific Python.

In [3]:
fname = "../data/inflammation-01.csv"
data = np.loadtxt(fname, delimiter=',')
print(data)

[[0. 0. 1. ... 3. 0. 0.]
 [0. 1. 2. ... 1. 0. 1.]
 [0. 1. 1. ... 2. 1. 1.]
 ...
 [0. 1. 1. ... 1. 1. 1.]
 [0. 0. 0. ... 0. 2. 0.]
 [0. 0. 1. ... 1. 1. 0.]]


The expression `np.loadtxt(...)` is a function call that asks Python to run the function `loadtxt` that belongs to the `numpy` library. This dotted notation is used everywhere in Python to refer to the parts of things as `thing.component` (see: [namespaces](https://stackoverflow.com/a/3913488/1063612)).

`numpy.loadtxt` has two arguments: the name of the file we want to read, and the delimiter that separates values on a line. These arguments need to be strings, so we put them in quotes (either `'` or `"`, it doesn't matter).

We saved the output of `loadtxt` to the variable `data`.
When we `print(data)`, only a few rows and columns are shown (with `...` to omit elements when displaying big arrays).
To save space, Python displays numbers as `1.` instead of `1.0` when there's nothing interesting after the decimal point.

### Other ways to load data from files

- `np.load`: Load arrays or [pickled](https://docs.python.org/3/library/pickle.html?highlight=pickle#module-pickle) objects from pickled files, saved using `np.save` with the extension `.npy` or `.npz` (the latter for gzip compressed files).
- `np.fromstring`: A new 1D array initialized from raw binary (`bytes`).
- `np.fromregex`: Construct an array from a text file, using regular expression parsing.
- [`np.genfromtxt`](https://docs.scipy.org/doc/numpy-dev/user/basics.io.genfromtxt.html): provides more sophisticated handling of, e.g. lines with missing values.

There are some more special I/O functions in [scipy.io](https://docs.scipy.org/doc/scipy/reference/io.html), for example for reading MATLAB data files and audio files, and [imageio](http://imageio.github.io) for reading image files.

## Manipulating Data

Now that our data is in memory, we can start doing things with it. First, let's ask what type of thing data refers to:

In [5]:
print(type(data))

<class 'numpy.ndarray'>


The output tells us that data currently refers to an N-dimensional array created by the NumPy library. We can see what its shape is like this:

In [9]:
print(data.shape)
n_patients, n_days = data.shape

(60, 40)


This tells us that `data` has 60 rows and 40 columns, which are 60 patients and 40 days. `data.shape` is a member of `data`, i.e. a value that is stored as part of an object.
We use the same dotted notation for the members of objects that we use for the functions in libraries because they have the same part-and-whole relationship.

If we want to get a single value from the matrix, we must provide an index in square brackets, just as we do with a `list`, but with as many indices as the number of dimensions in `shape` (two in this case):

In [10]:
print("first value in data", data[0,0])
print("middle value in data:", data[30, 20])

first value in data 0.0
middle value in data: 13.0


The expression `data[30, 20]` may not surprise you, but `data[0, 0]` might. Programming languages like Fortran and MATLAB start counting at 1, because that's what human beings have done for thousands of years. Languages in the C family (including C++, Java, Perl, and Python) count from 0 because that's simpler for computers to do. Just like with `list` and `str`, if we have an M×N array in Python, its indices go from 0 to M-1 on the first axis and 0 to N-1 on the second. It takes a bit of getting used to, but one way to remember the rule is that the index is how many steps we have to take from the start to get the item we want.

> **In the Corner.**
> What may also surprise you is that when Python displays an array, it shows the element with index [0, 0] in the upper left corner rather than the lower left. This is consistent with the way mathematicians draw matrices, but different from the Cartesian coordinates. The indices are (row, column) instead of (column, row) for the same reason, which can be confusing when plotting data.

An index like `[30, 20]` selects a single element of an array, but we can select whole sections as well. For example, we can select the first ten days (columns) of values for the first four (rows) patients like this:

In [14]:
print(data[0:4, 0:10])

[[0. 0. 1. 3. 1. 2. 4. 7. 8. 3.]
 [0. 1. 2. 1. 2. 1. 3. 2. 2. 6.]
 [0. 1. 1. 3. 3. 2. 6. 2. 5. 9.]
 [0. 0. 2. 0. 4. 2. 2. 1. 6. 7.]]


The slice `0:4` means, "Start at index 0 and go up to, but not including, index 4." Again, the up-to-but-not-including takes a bit of getting used to, but the rule is that the difference between the upper and lower bounds is the number of values in the slice.

We don't have to start slices at 0:

In [15]:
print(data[5:10, 0:10])

[[0. 0. 1. 2. 2. 4. 2. 1. 6. 4.]
 [0. 0. 2. 2. 4. 2. 2. 5. 5. 8.]
 [0. 0. 1. 2. 3. 1. 2. 3. 5. 3.]
 [0. 0. 0. 3. 1. 5. 6. 5. 5. 8.]
 [0. 1. 1. 2. 1. 3. 5. 3. 5. 8.]]


We also don't have to include the upper and lower bound on the slice. If we don't include the lower bound, Python uses 0 by default; if we don't include the upper, the slice runs to the end of the axis, and if we don't include either (i.e. if we just use ':' on its own), the slice includes everything:

In [17]:
small = data[:3, 36:].copy()
print('small is:')
print(small)

small is:
[[2. 3. 0. 0.]
 [1. 1. 0. 1.]
 [2. 2. 1. 1.]]


## Exercise 1

1. Print the last value of the array, that is, the value at the last row and last column.
1. Print the entire last row.
1. Print the entire last column.

0.0
[ 0.  0.  1.  0.  3.  2.  5.  4.  8.  2.  9.  3.  3. 10. 12.  9. 14. 11.
 13.  8.  6. 18. 11.  9. 13. 11.  8.  5.  5.  2.  8.  5.  3.  5.  4.  1.
  3.  1.  1.  0.]
[0. 1. 1. 1. 1. 1. 1. 1. 0. 0. 1. 1. 1. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0.
 1. 0. 1. 1. 0. 1. 1. 0. 1. 1. 1. 0. 1. 0. 1. 1. 1. 0. 0. 1. 1. 0. 0. 1.
 0. 1. 0. 0. 1. 1. 1. 1. 1. 1. 0. 0.]


## Operations

We can also perform common arithmetic operations on arrays: add, subtract, multiply, divide, etc.
When you perform these operations on arrays, the operation is done on each individual element of the array, i.e. elementwise.

In [19]:
doubledata = data * 2.0

will create a new array `doubledata` whose elements have the value of two times the value of the corresponding elements in `data`.

In [20]:
print('original:')
print(data[:3, 36:])
print('doubledata:')
print(doubledata[:3, 36:])

original:
[[2. 3. 0. 0.]
 [1. 1. 0. 1.]
 [2. 2. 1. 1.]]
doubledata:
[[4. 6. 0. 0.]
 [2. 2. 0. 2.]
 [4. 4. 2. 2.]]


This is also much faster than doing it with vanilla Python (`%timeit` is a magic command for measuring running time of single lines; use `%%timeit` to measure time of a whole cell).

In [21]:
data.dtype

dtype('float64')

In [29]:
n = 100000
%timeit [x**2 for x in range(n)]
%timeit np.arange(n)**2

30.8 ms ± 3.04 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
94 µs ± 5.24 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


We can also use binary (i.e. with two arguments) arithmetic operations like addition:

In [22]:
tripledata = doubledata + data

will give you an array where `tripledata[0,0]` will equal `doubledata[0,0]` plus `data[0,0]`, and so on for all other elements of the arrays.

In [23]:
print('tripledata:')
print(tripledata[:3, 36:])

tripledata:
[[6. 9. 0. 0.]
 [3. 3. 0. 3.]
 [6. 6. 3. 3.]]


Just another comparison:

In [32]:
n = 10000
%timeit [x + x**0.5 for x in range(n)]
%timeit x = np.arange(n); x + x**0.5

1.46 ms ± 53 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
259 µs ± 19.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


## Exercise 2

Calculate the square root of the data using `numpy`. 
Print the result for the first 5 columns of the first 3 rows.

[[0.         0.         1.         1.73205081 1.        ]
 [0.         1.         1.41421356 1.         1.41421356]
 [0.         1.         1.         1.73205081 1.73205081]]


## Getting help

You can try:

In [26]:
np.arange?

In [80]:
np.lookfor('zeros')

Search results for 'zeros'
--------------------------
numpy.zeros
    Return a new array of given shape and type, filled with zeros.
numpy.eye
    Return a 2-D array with ones on the diagonal and zeros elsewhere.
numpy.tri
    An array with ones at and below the given diagonal and zeros elsewhere.
numpy.trim_zeros
    Trim the leading and/or trailing zeros from a 1-D array or sequence.
numpy.zeros_like
    Return an array of zeros with the same shape and type as a given array.
numpy.ma.zeros
    Return a new array of given shape and type, filled with zeros.
numpy.matlib.zeros
    Return a matrix of given shape and type, filled with zeros.
numpy.matlib.eye
    Return a matrix with ones on the diagonal and zeros elsewhere.
numpy.bytes0.zfill
    B.zfill(width) -> copy of B
numpy.str0.zfill
    S.zfill(width) -> str
numpy.chararray.zfill
    Return the numeric string left-filled with zeros in a string of
numpy.polynomial.Hermite._roots
    Compute the roots of a Hermite series.
numpy.poly

In [81]:
np.concat*?

## Descriptive statistics

Often, we want to do more than add, subtract, multiply, and divide values of data. 
We can also do descriptive statistics on arrays.
If we want to find the average inflammation for all patients on all days, for example, we can just calculate the mean vakue of the array.

In [16]:
data.mean()

6.14875

`mean` is a method of the array, i.e. a function that belongs to it in the same way that the member shape does. If variables are nouns, methods are verbs: they describe operations that can be perfomed on the object.
This is why `data.shape` doesn't need to be called (it's a member, not a method) but `data.mean()` does (it's a method).
It is also why we need empty parentheses for `data.mean()`: even when we're not passing in any arguments, parentheses are how we tell Python to call a function (what would happen if you just use `data.mean`?).

NumPy arrays have lots of useful methods:

In [17]:
print('maximum inflammation:', data.max())
print('minimum inflammation:', data.min())
print('standard deviation:', data.std())

maximum inflammation: 20.0
minimum inflammation: 0.0
standard deviation: 4.613833197118566


When analyzing data, though, we often want to look at marginal statistics, such as the maximum value per patient or the average value per day.
One way to do this is to select the data we want to create a new temporary array, then ask it to do the calculation:

In [18]:
patient_0 = data[0, :] # 0 on the first axis, everything on the second
print('maximum inflammation for patient 0:', patient_0.max())

maximum inflammation for patient 0: 18.0


What if we need the maximum inflammation for all patients, or the average for each day? As the diagram below shows, we want to perform the operation across an axis:
![axis example](https://github.com/swcarpentry/python-novice-inflammation/raw/gh-pages/fig/python-operations-across-axes.png)
To support this, most array methods allow us to specify the axis we want to work on.
If we ask for the average across axis 0, we get:

In [27]:
print(data.mean(axis=0))

[ 0.          0.45        1.11666667  1.75        2.43333333  3.15
  3.8         3.88333333  5.23333333  5.51666667  5.95        5.9
  8.35        7.73333333  8.36666667  9.5         9.58333333 10.63333333
 11.56666667 12.35       13.25       11.96666667 11.03333333 10.16666667
 10.          8.66666667  9.15        7.25        7.33333333  6.58333333
  6.06666667  5.95        5.11666667  3.6         3.3         3.56666667
  2.48333333  1.5         1.13333333  0.56666667]


As a quick check, we can check the shape of the result:

In [29]:
print(data.mean(axis=0).shape)

(40,)


The expression `(40,)` tells us we have an 1D array of length 40, so this is the average inflammation per day for all patients. If we average across axis 1, we get:

In [22]:
print(data.mean(axis=1))

[5.45  5.425 6.1   5.9   5.55  6.225 5.975 6.65  6.625 6.525 6.775 5.8
 6.225 5.75  5.225 6.3   6.55  5.7   5.85  6.55  5.775 5.825 6.175 6.1
 5.8   6.425 6.05  6.025 6.175 6.55  6.175 6.35  6.725 6.125 7.075 5.725
 5.925 6.15  6.075 5.75  5.975 5.725 6.3   5.9   6.75  5.925 7.225 6.15
 5.95  6.275 5.7   6.1   6.825 5.975 6.725 5.7   6.25  6.4   7.05  5.9  ]


which is the average inflammation per patient across all days.

## Exercise 3

On which day did each patient had the most inflammation?
Use `data.argmax` to find out.

[19 20 20 20 19 18 21 20 18 20 19 22 17 19 17 18 21 21 17 21 21 22 18 19
 20 19 22 23 20 18 19 20 15 25 19 23 18 18 20 18 20 20 21 19 21 18 20 15
 24 18 19 20 21 19 18 17 23 19 22 21]


# Creating arrays

There are [5 general mechanisms for creating arrays](https://docs.scipy.org/doc/numpy-dev/user/basics.creation.html):

1. Conversion from other Python structures (e.g., lists, tuples)
1. Intrinsic numpy array creation objects (e.g., `arange`, `ones`, `zeros`, etc.)
1. Reading arrays from disk, either from standard or custom formats
1. Creating arrays from raw bytes through the use of strings or buffers
1. Use of special library functions (e.g., `numpy.random`)


Let's start by pecifying a list or list of lists to the `np.array` function:

In [25]:
a = np.array([0, 1, 2, 3])
print(a)
type(a), a.dtype

[0 1 2 3]


(numpy.ndarray, dtype('int64'))

The `dtype` attribute gives the data-type. 

We can force a specific data-type:

In [26]:
a = np.uint64([0, 1, 2, 3])
print(a)
type(a), a.dtype

[0 1 2 3]


(numpy.ndarray, dtype('uint64'))

In [27]:
a = np.float16([0, 1, 2, 3])
print(a)
type(a), a.dtype

[0. 1. 2. 3.]


(numpy.ndarray, dtype('float16'))

NumPy has many [data-types](https://docs.scipy.org/doc/numpy-dev/user/basics.types.html), we'll focus on the default `int` and `float` today.

We can create a 2D array from nested lists - make sure that all nested lists have the same length.

In [28]:
b = np.array(
    [
        [0, 1, 2], 
        [3, 4, 5]
    ]
)
print(b)

[[0 1 2]
 [3 4 5]]


In [29]:
c = np.array(
    [
        [
            [1], 
            [2]
        ], 
        [
            [3], 
            [4]
        ]
    ]
)
print(c)

[[[1]
  [2]]

 [[3]
  [4]]]


Arrays are N-dimensional, so you can specify how many dimensions that you would like:

In [30]:
d = np.array(
    [
        [
            [1, 2],
            [3, 4],
        ],
        [
            [5, 6],
            [7, 8],
        ],        
    ]
)
print(d)

[[[1 2]
  [3 4]]

 [[5 6]
  [7 8]]]


Check the number of dimensions and the shape:

In [31]:
print(d.ndim)
print(d.shape)

3
(2, 2, 2)


Use `np.arange`, whish is similar to `range`, but also accepts `float`s:

In [32]:
a = np.arange(10)  # end (exclusive)
print(a)

[0 1 2 3 4 5 6 7 8 9]


In [33]:
b = np.arange(-1.5, 9.5, 0.2) # start, end (exclusive), step
print(b)

[-1.5 -1.3 -1.1 -0.9 -0.7 -0.5 -0.3 -0.1  0.1  0.3  0.5  0.7  0.9  1.1
  1.3  1.5  1.7  1.9  2.1  2.3  2.5  2.7  2.9  3.1  3.3  3.5  3.7  3.9
  4.1  4.3  4.5  4.7  4.9  5.1  5.3  5.5  5.7  5.9  6.1  6.3  6.5  6.7
  6.9  7.1  7.3  7.5  7.7  7.9  8.1  8.3  8.5  8.7  8.9  9.1  9.3]


`np.linspace` is similar, but it accepts the required number of points rather than the required step.

In [34]:
c = np.linspace(0, 1, 6)   # start, end, num-points
print(c)

[0.  0.2 0.4 0.6 0.8 1. ]


Say we want to produce an array of the squares of numbers from 0 to 99.
There are a bunch of ways to do it using list comprehensions, `np.arange`, and using different approached for squaring the numbers. 

**Note**: currently NumPy doesn't use generator expressions for creating an array, so `np.array(x**2 for x in range(n))` doesn't work. There is an open [issue](https://github.com/numpy/numpy/pull/5863) on this.

Let's compare the different approaches in terms of running time:

In [50]:
n = 100000
%timeit np.array([x**2 for x in range(n)])
%timeit np.array([x**2 for x in np.arange(n)])
%timeit np.power(np.arange(n), 2)
%timeit np.power(range(n), 2)
%timeit np.arange(n)**2

32.8 ms ± 701 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
26.6 ms ± 1.06 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
253 µs ± 2.97 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
12.5 ms ± 1.07 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
92.4 µs ± 1.08 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


The fastest way to do it was also the most elegant, `np.arange(n)**2`, which makes us happy because:

> Beautiful is better than ugly.
Simple is better than complex.
Flat is better than nested.
Readability counts.
There should be one-- and preferably only one --obvious way to do it.

## Exercise 4

Create an array with the inverse ($1/x$) of the even numbers lower than or equal to 100.

[0.5        0.25       0.16666667 0.125      0.1        0.08333333
 0.07142857 0.0625     0.05555556 0.05       0.04545455 0.04166667
 0.03846154 0.03571429 0.03333333 0.03125    0.02941176 0.02777778
 0.02631579 0.025      0.02380952 0.02272727 0.02173913 0.02083333
 0.02       0.01923077 0.01851852 0.01785714 0.01724138 0.01666667
 0.01612903 0.015625   0.01515152 0.01470588 0.01428571 0.01388889
 0.01351351 0.01315789 0.01282051 0.0125     0.01219512 0.01190476
 0.01162791 0.01136364 0.01111111 0.01086957 0.0106383  0.01041667
 0.01020408 0.01      ]


## Creating arrays - continued
You can can create an empty array of a certain shape (which is given as a `tuple`) or with the same shape as another array; of course, the array will not actually be empty, but rather will have some arbitrary values as it will not be initialized.

In [36]:
d = np.empty((2, 4))
print(d)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [37]:
f = np.empty_like(d)
print(f)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]]


You can create an array full of 1s or 0s, or any single number:

In [38]:
a = np.ones((3, 3))
print(a)

[[1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]]


In [39]:
a = 5.134 * np.ones((3, 3))
print(a)

[[5.134 5.134 5.134]
 [5.134 5.134 5.134]
 [5.134 5.134 5.134]]


In [40]:
b = np.zeros((2, 2))
print(b)

[[0. 0.]
 [0. 0.]]


Create the identity matrix:

In [41]:
c = np.eye(3)
print(c)

[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


Create matrices by specifying the digonals:

In [42]:
d = np.diag([1, 2, 3, 4])
print(d)

[[1 0 0 0]
 [0 2 0 0]
 [0 0 3 0]
 [0 0 0 4]]


In [43]:
d = np.diag([1, 2, 3], 1) + np.diag([4, 5, 6], -1)
print(d)

[[0 1 0 0]
 [4 0 2 0]
 [0 5 0 3]
 [0 0 6 0]]


Create matrices by reshaping another matrix or array:

In [44]:
f = d.reshape((2, 8))
print(f)

[[0 1 0 0 4 0 2 0]
 [0 5 0 3 0 0 6 0]]


## Exercise 5

Skim through the documentation for `np.ravel`, and use this function to construct the array:
```py
[1 0 0 0 1 0 0 0 1]
```

[1 0 0 0 1 0 0 0 1]


## Random arrays

We'll set the random seed for reproducability (i.e. to get the same result every time), but in real-life application you should think if you want to set the seed.

In [55]:
np.random.seed(1231410) 

Start with drawing a single random number uniformly between 0 and 1:

In [56]:
np.random.random()

0.09823820032265207

An array of four random numbers between 0 and 1:

In [57]:
a = np.random.random(size=4)
print(a)

[0.21876966 0.40108675 0.26202072 0.66656901]


A 3x3 matrix or random numbers drawn from a normal distribution with mean 1 and standard deviation 0.5:

In [58]:
b = np.random.normal(1, 0.5, size=(3, 3))
print(b)

[[1.26338629 1.14548389 1.13313127]
 [0.26813755 1.01098859 0.7670454 ]
 [0.8370115  0.4950699  1.3018399 ]]


Now draw a 3x2x4 array from a Poisson distribution with mean 5:

In [59]:
c = np.random.poisson(5, size=(3, 2, 4))
print(c)

[[[6 5 8 7]
  [0 3 4 3]]

 [[4 5 6 8]
  [7 4 6 5]]

 [[3 7 4 4]
  [5 6 1 6]]]


## Exercise 6

1) Create a 4x5x6 array with numbers drawn from a geometric distribution with `p=0.1` (the number of trails until success, where the probability of success is `p`).

In [60]:
print(np.random.geometric(p=0.1, size=(4, 5, 6)))

[[[23  6  7  6  9  6]
  [16 10  6  7 30 21]
  [ 4  8  5 22  5 26]
  [23 17 70  5 11  1]
  [13  4  1  3 15  3]]

 [[ 4  1  4  3  9 15]
  [ 1  4  1  1  1 11]
  [ 1  4  3  3  4  1]
  [10  2  2  5  2  8]
  [ 9  4 13  8  3 25]]

 [[12  2  1  9 16  2]
  [18 13  9  5 11 25]
  [ 6  1 30 45  5  1]
  [17  5  1  9 11  5]
  [19  5  9  6  2 25]]

 [[11  1  5  6 13  4]
  [ 5  6  8 17  9  6]
  [ 1  8 19  7  6  5]
  [ 1  7  6  4  7  6]
  [ 7  3  8  7  2  8]]]


2) Normalize a 5x5 random matrix - that is, first subsctract by the minimum and then divide by the new maximum.

In [64]:
Z = np.random.random((5, 5))
print(Z)

[[0.50644665 0.09067333 0.49382812 0.55837365 0.94736843]
 [0.03450815 0.3672204  0.74196981 0.97348306 0.95168311]
 [0.81346559 0.75754522 0.92730313 0.34878739 0.82317238]
 [0.54976526 0.43255326 0.90928957 0.99930483 0.34100033]
 [0.58543851 0.76998339 0.60749917 0.83364564 0.3636184 ]]


In [65]:


print(Z)

[[0.4891585  0.05821453 0.47607955 0.5429802  0.94616855]
 [0.         0.34485219 0.73327539 0.97323605 0.95064066]
 [0.80737989 0.74941911 0.92537111 0.3257466  0.81744086]
 [0.53405771 0.4125689  0.90670028 1.         0.31767541]
 [0.5710326  0.76231112 0.5938982  0.82829627 0.34111876]]


# Broadcasting

A very powerful mechanism of NumPy arrays is [broadcasting](https://docs.scipy.org/doc/numpy/user/basics.broadcasting.html).
Broadcasting is used when an operation is used on two arrays of different shapes.
The rules are:

1. If arrays dimension differ, left-pad the smaller array's shape with 1s.
1. If the shapes differ, change any dimension of size 1 to match the dimension of the other array.
1. If shapes still differ, raise an error.

Some exmaples:
![broadcasting examples](http://www.astroml.org/_images/fig_broadcast_visual_1.png)

In [66]:
np.arange(3) + 5

array([5, 6, 7])

In [67]:
np.ones((3,3)) + np.arange(3)

array([[1., 2., 3.],
       [1., 2., 3.],
       [1., 2., 3.]])

In [68]:
np.arange(3).reshape((3, 1)) + np.arange(3)

array([[0, 1, 2],
       [1, 2, 3],
       [2, 3, 4]])

In [69]:
np.ones((3,3)) + np.ones((3,2))

ValueError: operands could not be broadcast together with shapes (3,3) (3,2) 

In [71]:
np.ones((3,3,1)) + np.ones((3,1,3))

array([[[2., 2., 2.],
        [2., 2., 2.],
        [2., 2., 2.]],

       [[2., 2., 2.],
        [2., 2., 2.],
        [2., 2., 2.]],

       [[2., 2., 2.],
        [2., 2., 2.],
        [2., 2., 2.]]])

## Exercise

Given a 1D array `X`, calculate the differences between each two elements of `X` using broadcasting and save it to array `D`.

In [72]:
X = np.linspace(0, 1, 50)

In [77]:
# your code here

In [78]:
assert D.shape == (50, 50)
assert (D.diagonal() == 0).all()
assert (D[5,5] == D[-5,-5])

# Indexing and slicing

[Indexing and slicing](https://docs.scipy.org/doc/numpy-dev/user/basics.indexing.html) on 1D arrays is similar to Python lists:

In [79]:
a = np.arange(1, 10)
print(a)
print(a[3])
print(a[2:5])
print(a[-2:-5:-1])

[1 2 3 4 5 6 7 8 9]
4
[3 4 5]
[8 7 6]


However, inconsistent with lists, array slicing returns a **view** rather then a copy, so **changing a slices changes the original array**:

In [80]:
print("Lists:")
a = [1, 2, 3, 4, 5, 6]
b = a[2:5]
print('a is b?', a is b)
print(a)
b[0] = 0
print(a)

Lists:
a is b? False
[1, 2, 3, 4, 5, 6]
[1, 2, 3, 4, 5, 6]


In [81]:
print("Arrays:")
a = np.arange(1, 7)
b = a[2:5]
print('a is b?', a is b)
print(a)
b[0] = 0
print(a)

Arrays:
a is b? False
[1 2 3 4 5 6]
[1 2 0 4 5 6]


This is useful if you want to save space and the CPU required by copying arrays, but it can also be dangerous.
If you explicitly want a **copy** rather than a **view**, call the `copy` method.

In [82]:
print("Arrays:")
a = np.arange(1, 7)
b = a[2:5].copy()
print('a is b?', a is b)
print(a)
b[0] = 0
print(a)

Arrays:
a is b? False
[1 2 3 4 5 6]
[1 2 3 4 5 6]


Arrays support multidimensional indexing and slicing:

In [83]:
a = np.diag([1,2,3])
print(a)
print()
print(a[:2,1:])

[[1 0 0]
 [0 2 0]
 [0 0 3]]

[[0 0]
 [2 0]]


In [84]:
y = np.arange(35).reshape(5,7)
print(y)

[[ 0  1  2  3  4  5  6]
 [ 7  8  9 10 11 12 13]
 [14 15 16 17 18 19 20]
 [21 22 23 24 25 26 27]
 [28 29 30 31 32 33 34]]


In [85]:
print(y[0])
print(y[0,:])

[0 1 2 3 4 5 6]
[0 1 2 3 4 5 6]


In [97]:
print(y[:,1])

[ 1  8 15 22 29]


Arrays can also be indexed using an array or list of indices, this is called **fancy indexing**:

In [98]:
a = np.arange(10, 30, 1)
print(a)
b = a[[1, 6, 9]]
print(b)
b = a[[1, 6, 9, 9, 9, 9, 9, 9]]
print(b)

[10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29]
[11 16 19]
[11 16 19 19 19 19 19 19]


In [86]:
y = np.arange(35).reshape(5,7)
print(y[[1,2,3]])

[[ 7  8  9 10 11 12 13]
 [14 15 16 17 18 19 20]
 [21 22 23 24 25 26 27]]


In [88]:
print(y[ [0, 2], [1, 2] ])

[ 1 16]


If one of the indexing lists is smaller than the other, NumPy will attempt broadcasting:

In [89]:
print(y[ [0, 2], [1] ])

[ 1 15]


But broadcasting isn't always possible:

In [90]:
y[[0,2,4], [0,1]]

IndexError: shape mismatch: indexing arrays could not be broadcast together with shapes (3,) (2,) 

The broadcasting mechanism permits index arrays to be combined with scalars for other indices. The effect is that the scalar value is used for all the corresponding values of the index arrays:

In [91]:
print(y[[0,2,4], 1])

[ 1 15 29]


## Boolean or mask arrays

You can create boolean arrays by using the comparison operators:

In [92]:
a = np.random.random(size=(4, 4))
print(a)
b = a < 0.5
print(b)

[[0.52379801 0.12082363 0.03355173 0.31833927]
 [0.98535473 0.24316266 0.02555671 0.22996596]
 [0.36768038 0.23910755 0.6506034  0.63901655]
 [0.56379706 0.24927365 0.04628701 0.38886171]]
[[False  True  True  True]
 [False  True  True  True]
 [ True  True False False]
 [False  True  True  True]]


These boolean arrays can be used for indexing:

In [93]:
c = np.arange(16).reshape((4, 4))
print(c)
print(c[c > 2])

[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]
 [12 13 14 15]]
[ 3  4  5  6  7  8  9 10 11 12 13 14 15]


## Exercise 8

Given a 1D array, negate (i.e. turn to negative) all elements which are between 3 and 8 (including both), in place (i.e. without creating a new array).

Note 1: you cannot use `and` because it acts on two booleans, whereas we want to do an elementwise "and". This is done using the operator `&`. 

Note 2: `&` is a Python bitwise operator which precedes arithmetic operators.

In [95]:
Z = np.arange(11)
print(Z)

[ 0  1  2  3  4  5  6  7  8  9 10]


In [97]:
# Your code here
print(Z)

[ 0  1  2 -3 -4 -5 -6 -7 -8  9 10]


# "Losing Your Loops": Fast Numerical Computing with NumPy 

From the PyCon 2015 conferece, a [presentation](https://speakerdeck.com/jakevdp/losing-your-loops-fast-numerical-computing-with-numpy-pycon-2015) by [Jake VanderPlas](http://vanderplas.com).

Also available on [YouTube](https://www.youtube.com/watch?v=EEUXKG97YRw).

In [106]:
from IPython.display import HTML
HTML('<script async class="speakerdeck-embed" data-id="a5d2540d0d4c452d91f8045ede6ca130" data-ratio="1.33333333333333" src="//speakerdeck.com/assets/embed.js"></script>')

# References

- [NumPy MedKit](http://mentat.za.net/numpy/numpy_advanced_slides/) for much more on indexing, slicing, and other advanced NumPy tricks.
- [NumPy tutorial](https://github.com/rougier/numpy-tutorial) with some exercises.
- [NumPy basics](https://docs.scipy.org/doc/numpy/user/basics.html)
- [NumPy for MATLAB users](https://docs.scipy.org/doc/numpy/user/numpy-for-matlab-users.html)
- [100 NumPy exercise](https://github.com/rougier/numpy-100)

# Colophon
This notebook was written by [Yoav Ram](http://python.yoavram.com) and is part of the [_Data Science with Python_](https://python.yoavram.com/Deep4Devs) workshops.

The notebook was written using [Python](http://python.org/) 3.6.5.
Dependencies listed in [environment.yml](../environment.yml).

This work is licensed under a CC BY-NC-SA 4.0 International License.

![Python logo](https://www.python.org/static/community_logos/python-logo.png)